In [1]:
#Package Imports

import pandas as pd
from pandasql import *
pysqldf = lambda q: sqldf(q, globals())

In [2]:
#Create Dataframe from u.data file
#Renaming item_id column in u.data to movie_id 

column_names = ['user_id','movie_id','rating','timestamp' ]
movie_ratings = pd.read_csv('u.data',delimiter='\t',names=column_names)
movie_ratings.head(5)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
#Create Dataframe from u.item file

column_names= ['movie_id','movie_title','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movie_data =  pd.read_csv('u.item',encoding="437",delimiter='|',names=column_names)
movie_data.head(5)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
#Create Dataframe from u.genre file
genre = pd.read_csv('u.genre',delimiter='|',names=['genere','id'])
genre.head(5)


,genere,id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [5]:
#Create Dataframe from u.user file
column_names = ['user_id','age','gender','occupation','zip_code']
user_data = pd.read_csv('u.user',delimiter='|',names=column_names)
user_data.head(5)

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


# Question:1
1. Find the Top 5 movies (by rating) in every genre

In [6]:
#Inner Join movie_data and movie_ratings based on movie_id  
dataset = movie_ratings.merge(movie_data,on='movie_id',how="inner")
dataset.head(3)

,user_id,movie_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
1,63,242,3,875747190,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
2,226,242,5,883888671,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# For genre Action
query="SELECT count(user_id) as counts,movie_title FROM (select * from dataset where Action=1 and rating = 5) group by movie_id order by counts desc limit 5"
pysqldf(query)

,counts,movie_title
0,325,Star Wars (1977)
1,214,"Godfather, The (1972)"
2,202,Raiders of the Lost Ark (1981)
3,179,Titanic (1997)
4,172,"Empire Strikes Back, The (1980)"


In [8]:
# Gives same result as above
query1= "select * from dataset where Action=1 and rating = 5";
df = pysqldf(query1)
query = "SELECT count(user_id) as counts,movie_title FROM df group by movie_id order by counts desc limit 5"
pysqldf(query)

,counts,movie_title
0,325,Star Wars (1977)
1,214,"Godfather, The (1972)"
2,202,Raiders of the Lost Ark (1981)
3,179,Titanic (1997)
4,172,"Empire Strikes Back, The (1980)"


In [9]:
# Gives same result as above
df = dataset[(dataset["Action"] == 1) & (dataset["rating"] == 5)]
query = "SELECT count(user_id) as counts,movie_title FROM df group by movie_id order by counts desc limit 5"
pysqldf(query)

,counts,movie_title
0,325,Star Wars (1977)
1,214,"Godfather, The (1972)"
2,202,Raiders of the Lost Ark (1981)
3,179,Titanic (1997)
4,172,"Empire Strikes Back, The (1980)"


In [10]:
# Snippet to get Top 5 Movies of all genres and saves to csv

genere = pd.read_csv('u.genre',delimiter='|',names=['genere','id'])
with open('FirstQuesAnswer.csv', 'a') as f:
    for i in genere['genere']:
        df=dataset[(dataset[str(i)] == 1) & (dataset["rating"] == 5)]
        query = "SELECT count(user_id) as counts,movie_title as 'Top5Moives:' FROM df group by movie_id order by counts desc limit 5"
        df=pysqldf(query)
        data=df.drop("counts",axis=1)
        f.write("-----------------------------------------\n")
        f.write("GenereName: "+i+"\n")
        data.to_csv(f, index=False)

# Question 2
2. Which genre of movies are most popular for each age group?

In [11]:
# Inner join dataset with u.user based on user_id
newdata = dataset.merge(user_data,on='user_id',how="inner")
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
newdata['age_group'] = pd.cut(newdata.age, range(0, 81, 10), right=False, labels=labels)
newdata.head(5)


,user_id,movie_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,...,Romance,Sci-Fi,Thriller,War,Western,age,gender,occupation,zip_code,age_group
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,49,M,writer,55105,40-49
1,196,257,2,881251577,Men in Black (1997),04-Jul-1997,NaN,http://us.imdb.com/M/title-exact?Men+in+Black+...,0,1,...,0,1,0,0,0,49,M,writer,55105,40-49
2,196,111,4,881251793,"Truth About Cats & Dogs, The (1996)",26-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Truth%20About...,0,0,...,1,0,0,0,0,49,M,writer,55105,40-49
3,196,25,4,881251955,"Birdcage, The (1996)",08-Mar-1996,NaN,"http://us.imdb.com/M/title-exact?Birdcage,%20T...",0,0,...,0,0,0,0,0,49,M,writer,55105,40-49
4,196,382,4,881251843,"Adventures of Priscilla, Queen of the Desert, ...",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Adventures%20...,0,0,...,0,0,0,0,0,49,M,writer,55105,40-49


In [12]:
#grouped by age
query = "SELECT age ,sum(Action) as Action ,sum(Adventure) as Adventure,sum(Animation) as Animation,sum(Comedy) as Comedy,sum(Crime) as Crime ,sum(Documentary) as Documentary,sum(Drama) as Drama,sum(Fantasy) as Fantasy,sum(Horror) as Horror,sum(Musical) as Musical,sum(Mystery) as Mystery FROM newdata group by age"
groupedbyage = pysqldf(query)
groupedbyage = groupedbyage.set_index('age')
df = groupedbyage['PopularGenre'] = groupedbyage.idxmax(axis=1)
df.head(5)

age
7        Comedy
10    Animation
11       Action
13       Comedy
14        Drama
dtype: object

In [13]:
#grouped by agegroup
query = "SELECT  age_group,sum(Action) as Action ,sum(Adventure) as Adventure,sum(Animation) as Animation,sum(Comedy) as Comedy,sum(Crime) as Crime ,sum(Documentary) as Documentary,sum(Drama) as Drama,sum(Fantasy) as Fantasy,sum(Horror) as Horror,sum(Musical) as Musical,sum(Mystery) as Mystery FROM newdata group by age_group"
groupedbyage = pysqldf(query)

# Final answer save to CSV
groupedbyage = groupedbyage.set_index('age_group')
df = groupedbyage['PopularGenre'] = groupedbyage.idxmax(axis=1)
df.to_csv('secondQuesAnswer.csv', index=True,header=True)

# Question 3

3. Which genre of movies are most rated (not necessarily most popular) for each occupation?


In [14]:
query = "SELECT occupation ,sum(Action) as Action ,sum(Adventure) as Adventure,sum(Animation) as Animation,sum(Comedy) as Comedy,sum(Crime) as Crime ,sum(Documentary) as Documentary,sum(Drama) as Drama,sum(Fantasy) as Fantasy,sum(Horror) as Horror,sum(Musical) as Musical,sum(Mystery) as Mystery FROM newdata group by occupation"
groupedbyoccupation = pysqldf(query)
groupedbyoccupation = groupedbyoccupation.set_index('occupation')
df= groupedbyoccupation['Max'] = groupedbyoccupation.idxmax(axis=1)
df.to_csv('thirdQuesAnswer.csv', index=True,header=True)